In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [2]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

input_file = "train.csv"
df_train = pd.read_csv(input_file)

data_train = shuffle(df_train, random_state=42)

In [3]:
X_train = data_train.drop(["Activity"], axis=1)

X_train

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject
4525,0.283203,-0.047024,-0.168986,0.384949,0.176898,-0.310332,0.381757,0.122611,-0.332984,0.465563,...,-0.730593,-0.932568,-0.034924,0.558036,0.258975,-0.854858,-0.784330,0.222960,-0.066506,22
1446,0.256904,-0.036623,-0.133856,0.201409,-0.154142,0.344183,0.092771,-0.182114,0.291899,0.586004,...,-0.032958,-0.338095,0.017986,-0.475545,0.942947,-0.567147,-0.665156,0.178975,0.243362,7
5995,0.291316,-0.001065,-0.072461,-0.336609,-0.279162,-0.303323,-0.381421,-0.253026,-0.337381,0.074430,...,-0.351997,-0.698254,-0.044663,-0.551806,-0.680774,0.161405,-0.857202,0.192929,0.046716,27
4222,0.276116,-0.010909,-0.102886,-0.992196,-0.982169,-0.981127,-0.992838,-0.981366,-0.979864,-0.931924,...,-0.623358,-0.906098,-0.177275,-0.309304,0.176515,-0.143818,-0.625215,-0.106678,-0.147469,21
6754,0.256382,0.000428,-0.113664,0.075014,0.046502,-0.369482,-0.024794,-0.085337,-0.385018,0.562230,...,0.020630,-0.316113,0.347143,0.590475,0.831084,0.465453,-0.860617,0.150598,-0.067109,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5191,0.278897,-0.030306,-0.096043,-0.555352,-0.104055,-0.438064,-0.572530,-0.112149,-0.429688,-0.479744,...,0.417731,0.200267,-0.155279,-0.137981,0.929236,-0.583898,-0.703989,0.159112,-0.185944,25
5226,0.289183,-0.049248,-0.125083,-0.290043,-0.212102,-0.469731,-0.307317,-0.209558,-0.528635,-0.283244,...,-0.021970,-0.360982,-0.019533,-0.896005,-0.039591,0.494178,-0.670139,0.106156,-0.220746,25
5390,0.293946,-0.018341,-0.119916,-0.627198,-0.216566,-0.424764,-0.648666,-0.253814,-0.417569,-0.547082,...,0.527152,0.348736,-0.414621,-0.488542,0.095895,-0.297162,-0.856512,0.183547,-0.034939,25
860,0.280475,-0.018976,-0.113756,-0.994825,-0.985314,-0.965857,-0.995170,-0.984285,-0.963293,-0.937503,...,-0.889671,-0.984051,-0.079515,0.014291,-0.290347,0.602344,-0.839693,0.081388,-0.098270,5


In [4]:
y_train = data_train["Activity"].copy()

y_train

4525    WALKING_DOWNSTAIRS
1446    WALKING_DOWNSTAIRS
5995               WALKING
4222               SITTING
6754    WALKING_DOWNSTAIRS
               ...        
5191               WALKING
5226    WALKING_DOWNSTAIRS
5390               WALKING
860               STANDING
7270               WALKING
Name: Activity, Length: 7352, dtype: object

In [5]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)

In [6]:
def build_model():
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[562,]),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(512, activation="selu"),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(512, activation="selu"),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(512, activation="selu"),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(512, activation="selu"),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(512, activation="selu"),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(6, activation="softmax")
    ])

    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    # loss = ???
    
    return model

In [7]:
def k_fold(num_epochs):
    k = 10
    num_val = len(X_train) // k
    all_scores = []
    num_epochs = num_epochs
    
    np.random.seed(42)
    tf.random.set_seed(42)
    
    for i in range(k):
        print('processing fold #', i)

        X_val = X_train[i * num_val: (i + 1) * num_val]
        y_val = y_train[i * num_val: (i + 1) * num_val]

        X_train_part = np.concatenate(
            [X_train[:i * num_val],
            X_train[(i + 1) * num_val:]],
            axis = 0)

        y_train_part = np.concatenate(
            [y_train[:i * num_val],
            y_train[(i + 1) * num_val:]],
            axis = 0)

        model = build_model()
        history = model.fit(X_train_part, y_train_part, 
                            epochs = num_epochs, verbose = 0)

        val_loss, val_acc = model.evaluate(X_val, y_val, verbose = 0)
        all_scores.append(val_acc)
        
    print(f"검증 결과 : {all_scores}")
    
    print(f"평균 정확도 : {np.mean(all_scores)}")

In [8]:
k_fold(370)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9
검증 결과 : [0.9972789287567139, 0.9904761910438538, 0.994557797908783, 0.9863945841789246, 0.9918367266654968, 0.994557797908783, 0.9959183931350708, 0.994557797908783, 0.9891156554222107, 0.9972789287567139]
평균 정확도 : 0.9931972801685334
